In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import random

In [ ]:
def brand_links(url):
    r = requests.get(url, headers = {'user-agent': 'Chrome/58.0.3029.110 accept-language'})
    soup = BeautifulSoup(r.content, 'html.parser')
    brand_list = ['alcatel', 'Apple' ,'Asus' ,'BLU' ,'HTC', 'Huawei', 'Infinix', 'Lenovo', 'LG', 'Nokia', 'Sony' , 'Xiaomi', 'ZTE', 'Samsung']
    brand_page_links = list()
    for row in soup.select('.st-text')[0].find_all('td'):
        for brand in brand_list:
            if brand in row.find('a').text:
                brand_page_links.append('https://www.gsmarena.com/' + row.find('a')['href'])
    brand_page_links.remove('https://www.gsmarena.com/garmin_asus-phones-65.php')
    brand_page_links.remove('https://www.gsmarena.com/sony_ericsson-phones-19.php')
    return brand_page_links

In [ ]:
brand_page_links = brand_links('https://www.gsmarena.com/makers.php3')

In [ ]:
brand_page_links

In [ ]:
def brand_pages(brand_page_links):
    link_list = list()
    for url in brand_page_links:
        r = requests.get(url, headers = {'user-agent': 'Chrome/58.0.3029.110 accept-language'})
        soup = BeautifulSoup(r.content, 'html.parser')
        print(url)
        try:
            max_num_page = int(soup.select('.nav-pages')[0].find_all('a')[3].text)
        except:
            max_num_page = int(soup.select('.nav-pages')[0].find_all('a')[2].text)
        first_sec, second_sec = url.split('-phones-')
        for num in range(1, max_num_page + 1):
            link_list.append(first_sec + '-phones-f-' + second_sec.split('.')[0] + '-0-p' + '{}'.format(num) + '.' + second_sec.split('.')[1])
    return link_list

In [ ]:
link_list = brand_pages(brand_page_links)

In [ ]:
def phone_links(link_list):
    phone_links = list()
    for url in link_list:
        print(url)
        r = requests.get(url, headers = {'user-agent': 'Chrome/58.0.3029.110 accept-language'})
        soup = BeautifulSoup(r.content, 'html.parser')
        for phone in soup.select('.makers')[0].find_all('li'):
            phone_links.append('https://www.gsmarena.com/' + phone.find('a')['href'])
    return phone_links

In [ ]:
phone_links = phone_links(link_list)

In [ ]:
phones_details = list()
core_dict = {
    'Octa':8,
    'Dual':2,
    'Quad':4,
    'Hexa':6,
    'Deca':10
}
sim_list = ['Micro-SIM', 'Nano-SIM', 'Mini-SIM']

for id, url in enumerate(phone_links):
    print(id + 1, url, 'started')
    phone = dict()
    r = requests.get(url, headers = {'user-agent': 'Chrome/58.0.3029.110 accept-language'})
    soup = BeautifulSoup(r.content, 'html.parser')
    headers = [row.text for row in soup.select('#specs-list')[0].find_all('th')]
    try:
        Network = headers.index('Network')
    except:
        Network = -1
        
    try:
        Launch = headers.index('Launch')
    except:
        Launch = -1
        
    try:
        Body = headers.index('Body')
    except:
        Body = -1
        
    try:
        Display = headers.index('Display')
    except:
        Display = -1
    try:
        Platform = headers.index('Platform')
    except:
        Platform = -1
        
    try:
        Memory = headers.index('Memory')
    except:
        Memory = -1
    try:
        Features = headers.index('Features')
    except:
        Features = -1
    try:
        Battery = headers.index('Battery')
    except:
        Battery = -1
    try:
        Misc = headers.index('Misc')
    except:
        Misc = -1
    
    try:
        year = int(re.findall(r'\d+', soup.select('#specs-list')[0].find_all('table')[Launch].find(class_='nfo').text)[0])
    except:
        year = 0
    if year < 2010:
        continue
    phone['id'] = id + 1
    phone['brand'] = soup.select('.specs-phone-name-title')[0].text.split(' ')[0]
    phone['name'] = ' '.join(soup.select('.specs-phone-name-title')[0].text.split(' ')[1:])
    phone['year'] = year
    if Platform != -1:
        platform_info = [row.text for row in soup.select('#specs-list')[0].find_all('table')[Platform].find_all('td')]
        try:
            os_index = platform_info.index('OS') + 1
        except:
            os_index = -1
        try:
            core_index = platform_info.index('CPU') + 1
        except:
            core_index = -1
        if core_index != -1:
            core = platform_info[core_index].split(' ')[0].split('-')[0]
            try:
                phone['core'] = float(core)
            except:
                phone['core'] = core_dict.get(core, '')
        else:
            phone['core'] = ''
        if os_index != -1:
            try:
                type = platform_info[os_index].split(',')[0].split(' ')[0]
                version = platform_info[os_index].split(',')[0].split(' ')[1]
            except:
                try:
                    type = platform_info[os_index].split(',')[0].split('^')[0]
                    version = platform_info[os_index].split(',')[0].split('^')[1]
                except:
                    type = platform_info[os_index].split(',')[0]
                    version = ''
            phone['type'] = type
            phone['version'] = version        
        else:
            phone['type'] = ''
            phone['version'] = ''           
    else:
        phone['core'] = ''
        phone['type'] = ''
        phone['version'] = ''
    if Memory != -1:
        memory_info = [row.text for row in soup.select('#specs-list')[0].find_all('table')[Memory].find_all('td')]
        try:
            memory_index = memory_info.index('Internal') + 1
        except:
            memory_index = -1
        if memory_index != -1:
            memory = list()
            items = memory_info[memory_index].split(', ')
            for item in items:
                hard_ram = item.split()[:2]
                hard_ram = [n.strip('GB') for n in hard_ram]
                hard_ram = [n.strip('T') for n in hard_ram]
                hard_ram = [n.strip('M') for n in hard_ram]
                memory.append(tuple(hard_ram))
            phone['memory'] = memory
        else:
            phone['memory'] = ''
    else :
        phone['memory'] = ''
    if Network != -1:
        two = '0'
        tree = '0'
        four = '0'
        for row in soup.select('#specs-list')[0].find_all('table')[Network].find_all('td'):
            if '2G' in row.text:
                two = '1'
            if '3G' in row.text:
                tree = '1'
            if '4G' in row.text:
                four = '1'
        phone['network'] = ''.join([four, tree, two])
    else:
        phone['network'] = '000'
    if Features != -1:
        sensors_str = soup.select('#specs-list')[0].find_all('table')[Features].find_all(class_='nfo')[0].text
        sensors_list = list()
        items = sensors_str.split(',')
        for item in items:
            item = item.strip()
            if '(' in item:
                item = item.split('(')[0]
            if ')' in item:
                continue
            item = item.strip()
            sensors_list.append(item)
        phone['sensor'] = sensors_list
    else:
        phone['sensor'] = ''
    if Body != -1:
        body_info = [row.text for row in soup.select('#specs-list')[0].find_all('table')[Body].find_all('td')]
        try:
            dimensions_index = body_info.index('Dimensions') + 1
        except:
            dimensions_index = -1   
        try:
            weigth_index = body_info.index('Weight') + 1
        except:
            weigth_index = -1   
        try:
            sim_index = body_info.index('SIM') + 1
        except:
            sim_index = -1
        try:
            phone['dimensions'] = body_info[dimensions_index].split(' mm')[0]
        except:
            phone['dimensions'] = ''
        try:
            phone['weight'] = body_info[weigth_index].split(' g')[0]
        except:
            phone['weigth'] = ''
        try:
            for sim in sim_list:
                if sim in body_info[sim_index]:
                    phone['sim'] = sim
                    break
        except:
            phone['sim'] = ''
    else:
        phone['dimensions'] = ''
        phone['weight'] = ''
        phone['sim'] = ''
    if Display != -1:
        display_info = [row.text for row in soup.select('#specs-list')[0].find_all('table')[Display].find_all('td')]
        try:
            size = display_info.index('Size') + 1
        except:
            size = -1
        if size != -1:
            try:
                phone['size'] = display_info[size].split()[0]
            except:
                phone['size'] = ''
            try:
                phone['screen_to_body_ratio'] = display_info[size].split('~')[1].split('%')[0]
            except:
                phone['screen_to_body_ratio'] = ''
        else:
            phone['size'] = ''
            phone['screen_to_body_ratio'] = ''
        try:
            resolution = display_info.index('Resolution') + 1
        except:
            resolution = -1
        if resolution != -1:
            phone['resolution'] = display_info[resolution].split(' pixels')[0]
            try:
                phone['ppi_density'] = display_info[resolution].split('~')[1].split(' ')[0]
            except:
                phone['ppi_density'] = ''
        else:
            phone['resolution'] = ''
            phone['ppi_density'] = ''
    else:
        phone['size'] = ''
        phone['screen_to_body_ratio'] = ''
        phone['resolution'] = ''
        phone['ppi_density'] = ''
    if Misc != -1:
        price_info = [row.text for row in soup.select('#specs-list')[0].find_all('table')[Misc].find_all('td')]
        try:
            price_index = price_info.index('Price') + 1
        except:
            price_index = -1
        if price_index != -1:
            if 'Prices' in [i.text for i in soup.select('.article-info-meta-link')]:
                try:
                    phone['price'] = price_info[price_index].split()[price_info[price_index].split().index('£') + 1].replace(',', '')
                except:
                    phone['price'] = ''
            else:
                phone['price'] = re.findall(r'\d+', price_info[price_index])[0]
        else:
            phone['price'] = ''
    else:
        phone['price'] = ''
    if Battery != -1:
        battery_info = [row.text for row in soup.select('#specs-list')[0].find_all('table')[Battery].find_all('td')]
        try:
            battery_index = battery_info.index('Type') + 1
        except:
            battery_index = -1
        try:
            phone['battery'] = re.findall(r'\d+', battery_info[battery_index])[0]
        except:
            phone['battery'] = ''
    else:
        phone['battery'] = ''
    phones_details.append(phone)
    print('finished')

In [ ]:
phones_details